In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

# Arguments

In [2]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN/128x128/'

In [3]:
pretrained = ''
resume = './log/pggan/checkpoint.pth.tar'

In [4]:
# Model
model_name = 'efficientnet-b3' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 256
test_batch = 200
lr = 0.04
schedule = [150, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

size = (128, 128)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 10.70M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in schedule:
        state['lr'] *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    adjust_learning_rate(optimizer, epoch)
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    state['lr'] = lr
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [8 | 400] LR: 0.236000
1/502 Data:1.308 | Batch:6.489 | Total:0:00:06 | ETA:0:54:12 | Loss:0.40490442514419556 | top1:83.203125
11/502 Data:0.009 | Batch:0.636 | Total:0:00:13 | ETA:0:10:31 | Loss:0.4280526773496108 | top1:79.90057373046875
21/502 Data:0.001 | Batch:0.631 | Total:0:00:20 | ETA:0:05:16 | Loss:0.4214275450933547 | top1:80.39434814453125
31/502 Data:0.001 | Batch:0.646 | Total:0:00:26 | ETA:0:05:17 | Loss:0.42317353237059807 | top1:80.2923355102539
41/502 Data:0.012 | Batch:0.676 | Total:0:00:33 | ETA:0:05:30 | Loss:0.42724718844018333 | top1:80.16387176513672
51/502 Data:0.001 | Batch:0.798 | Total:0:00:41 | ETA:0:05:15 | Loss:0.4261435054096521 | top1:80.26194763183594
61/502 Data:0.001 | Batch:0.680 | Total:0:00:48 | ETA:0:05:18 | Loss:0.4272739652727471 | top1:80.14215850830078
71/502 Data:0.008 | Batch:0.696 | Total:0:00:55 | ETA:0:05:03 | Loss:0.4293803073151011 | top1:80.02310943603516
81/502 Data:0.001 | Batch:0.811 | Total:0:01:03 | ETA:0:05:55 | Loss:0.4

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.000 | Batch:0.522 | Total:0:00:30 | ETA:0:00:00 | Loss:0.2008877910000513 | top1:99.34579467773438

Epoch: [9 | 400] LR: 0.068000
1/502 Data:1.009 | Batch:2.068 | Total:0:00:02 | ETA:0:17:17 | Loss:0.430498868227005 | top1:80.078125
11/502 Data:0.016 | Batch:0.758 | Total:0:00:08 | ETA:0:06:43 | Loss:0.41725732521577313 | top1:81.10795593261719
21/502 Data:0.008 | Batch:0.660 | Total:0:00:15 | ETA:0:05:35 | Loss:0.4136201427096412 | top1:81.21279907226562
31/502 Data:0.001 | Batch:0.630 | Total:0:00:22 | ETA:0:05:38 | Loss:0.41124791099179175 | top1:81.31300354003906
41/502 Data:0.010 | Batch:0.717 | Total:0:00:29 | ETA:0:05:15 | Loss:0.41041680516266243 | top1:81.40243530273438
51/502 Data:0.001 | Batch:0.659 | Total:0:00:36 | ETA:0:05:16 | Loss:0.4125130790121415 | top1:81.14276885986328
61/502 Data:0.001 | Batch:0.688 | Total:0:00:43 | ETA:0:04:45 | Loss:0.41160816833621167 | top1:81.20516967773438
71/502 Data:0.001 | Batch:0.631 | Total:0:00:50 | ETA:0:05:08 | Loss:0

191/502 Data:0.001 | Batch:0.700 | Total:0:02:18 | ETA:0:04:31 | Loss:0.4188146670763405 | top1:80.3358154296875
201/502 Data:0.001 | Batch:0.756 | Total:0:02:26 | ETA:0:04:09 | Loss:0.41877427786143856 | top1:80.32105255126953
211/502 Data:0.001 | Batch:0.821 | Total:0:02:34 | ETA:0:03:31 | Loss:0.4188301188968369 | top1:80.31139373779297
221/502 Data:0.001 | Batch:0.819 | Total:0:02:42 | ETA:0:03:44 | Loss:0.41824180818251355 | top1:80.35562896728516
231/502 Data:0.013 | Batch:0.823 | Total:0:02:49 | ETA:0:03:33 | Loss:0.41809360895838055 | top1:80.39942169189453
241/502 Data:0.014 | Batch:0.826 | Total:0:02:58 | ETA:0:03:37 | Loss:0.41791215452415814 | top1:80.42985534667969
251/502 Data:0.001 | Batch:0.664 | Total:0:03:05 | ETA:0:02:57 | Loss:0.4171913116576662 | top1:80.47808837890625
261/502 Data:0.009 | Batch:0.842 | Total:0:03:12 | ETA:0:02:52 | Loss:0.4171266360529538 | top1:80.503173828125
271/502 Data:0.006 | Batch:0.676 | Total:0:03:19 | ETA:0:02:45 | Loss:0.417044452745536

391/502 Data:0.001 | Batch:0.632 | Total:0:04:46 | ETA:0:01:17 | Loss:0.41694448183259697 | top1:80.45376586914062
401/502 Data:0.008 | Batch:0.705 | Total:0:04:52 | ETA:0:01:09 | Loss:0.41705136725730135 | top1:80.45219421386719
411/502 Data:0.001 | Batch:0.709 | Total:0:05:00 | ETA:0:01:11 | Loss:0.417202640333895 | top1:80.44783782958984
421/502 Data:0.007 | Batch:0.698 | Total:0:05:07 | ETA:0:00:57 | Loss:0.41736983743246264 | top1:80.42792510986328
431/502 Data:0.002 | Batch:0.790 | Total:0:05:15 | ETA:0:00:52 | Loss:0.4176934895963514 | top1:80.402587890625
441/502 Data:0.016 | Batch:0.653 | Total:0:05:21 | ETA:0:00:42 | Loss:0.41756901841044697 | top1:80.4244613647461
451/502 Data:0.001 | Batch:0.638 | Total:0:05:28 | ETA:0:00:35 | Loss:0.4173187029996098 | top1:80.43323516845703
461/502 Data:0.009 | Batch:0.722 | Total:0:05:35 | ETA:0:00:28 | Loss:0.4173905852042672 | top1:80.42892456054688
471/502 Data:0.016 | Batch:0.787 | Total:0:05:42 | ETA:0:00:23 | Loss:0.4172289460074876

...

471/502 Data:0.001 | Batch:0.631 | Total:0:05:05 | ETA:0:00:20 | Loss:0.18300124177127886 | top1:92.48938751220703
481/502 Data:0.001 | Batch:0.647 | Total:0:05:11 | ETA:0:00:14 | Loss:0.18332268787149084 | top1:92.47254943847656
491/502 Data:0.007 | Batch:0.650 | Total:0:05:18 | ETA:0:00:08 | Loss:0.18419236327140248 | top1:92.42935943603516
501/502 Data:0.000 | Batch:0.631 | Total:0:05:24 | ETA:0:00:01 | Loss:0.1846520040860909 | top1:92.408935546875
161/161 Data:0.000 | Batch:0.082 | Total:0:00:26 | ETA:0:00:00 | Loss:0.046308592542123946 | top1:98.88162231445312

Epoch: [141 | 400] LR: 0.252001
1/502 Data:0.887 | Batch:1.586 | Total:0:00:01 | ETA:0:13:15 | Loss:0.1560952067375183 | top1:92.578125
11/502 Data:0.001 | Batch:0.633 | Total:0:00:07 | ETA:0:06:00 | Loss:0.17515841939232565 | top1:92.47159576416016
21/502 Data:0.001 | Batch:0.638 | Total:0:00:14 | ETA:0:05:14 | Loss:0.17160884219975697 | top1:92.89434814453125
31/502 Data:0.001 | Batch:0.645 | Total:0:00:20 | ETA:0:05:06 

151/502 Data:0.001 | Batch:0.641 | Total:0:01:40 | ETA:0:03:50 | Loss:0.18914964869125 | top1:92.18232727050781
161/502 Data:0.001 | Batch:0.659 | Total:0:01:47 | ETA:0:03:42 | Loss:0.1885803973545199 | top1:92.19477844238281
171/502 Data:0.000 | Batch:0.625 | Total:0:01:53 | ETA:0:03:35 | Loss:0.18844517346536904 | top1:92.19892120361328
181/502 Data:0.001 | Batch:0.632 | Total:0:02:00 | ETA:0:03:27 | Loss:0.18790955615142432 | top1:92.19397735595703
191/502 Data:0.001 | Batch:0.631 | Total:0:02:06 | ETA:0:03:22 | Loss:0.1867498739461624 | top1:92.25294494628906
201/502 Data:0.001 | Batch:0.735 | Total:0:02:13 | ETA:0:03:15 | Loss:0.18822953286603908 | top1:92.1913833618164
211/502 Data:0.001 | Batch:0.631 | Total:0:02:19 | ETA:0:03:17 | Loss:0.1903117775422702 | top1:92.05791473388672
221/502 Data:0.001 | Batch:0.660 | Total:0:02:26 | ETA:0:03:01 | Loss:0.19014141513076843 | top1:92.0443344116211
231/502 Data:0.000 | Batch:0.661 | Total:0:02:32 | ETA:0:02:59 | Loss:0.1901534559342252

351/502 Data:0.001 | Batch:0.664 | Total:0:03:48 | ETA:0:01:43 | Loss:0.21004132567732423 | top1:91.24376678466797
361/502 Data:0.001 | Batch:0.673 | Total:0:03:55 | ETA:0:01:31 | Loss:0.21019388196623556 | top1:91.2450180053711
371/502 Data:0.001 | Batch:0.675 | Total:0:04:01 | ETA:0:01:25 | Loss:0.20931823220294762 | top1:91.29674530029297
381/502 Data:0.001 | Batch:0.636 | Total:0:04:08 | ETA:0:01:19 | Loss:0.20942199396414393 | top1:91.30167388916016
391/502 Data:0.001 | Batch:0.631 | Total:0:04:14 | ETA:0:01:11 | Loss:0.20895441370013426 | top1:91.3203353881836
401/502 Data:0.001 | Batch:0.629 | Total:0:04:21 | ETA:0:01:08 | Loss:0.20908515803757452 | top1:91.32442474365234
411/502 Data:0.001 | Batch:0.648 | Total:0:04:27 | ETA:0:01:00 | Loss:0.20912029391150114 | top1:91.31976318359375
421/502 Data:0.001 | Batch:0.631 | Total:0:04:34 | ETA:0:00:53 | Loss:0.2096534482338366 | top1:91.30883026123047
431/502 Data:0.001 | Batch:0.646 | Total:0:04:40 | ETA:0:00:46 | Loss:0.21098549541

31/502 Data:0.001 | Batch:0.639 | Total:0:00:21 | ETA:0:05:10 | Loss:0.19062310288990697 | top1:91.96068572998047
41/502 Data:0.001 | Batch:0.694 | Total:0:00:27 | ETA:0:04:56 | Loss:0.1984224499362271 | top1:91.66349029541016
51/502 Data:0.001 | Batch:0.645 | Total:0:00:34 | ETA:0:04:58 | Loss:0.20152709457804174 | top1:91.5517807006836
61/502 Data:0.001 | Batch:0.627 | Total:0:00:40 | ETA:0:04:43 | Loss:0.20303330863596963 | top1:91.4766845703125
71/502 Data:0.001 | Batch:0.646 | Total:0:00:47 | ETA:0:04:35 | Loss:0.20325561738769773 | top1:91.50527954101562
81/502 Data:0.001 | Batch:0.628 | Total:0:00:53 | ETA:0:04:32 | Loss:0.20488954427433603 | top1:91.40625
91/502 Data:0.001 | Batch:0.638 | Total:0:00:59 | ETA:0:04:23 | Loss:0.20588649997671882 | top1:91.4105453491211
101/502 Data:0.001 | Batch:0.667 | Total:0:01:06 | ETA:0:04:19 | Loss:0.20717301396745266 | top1:91.34823608398438
111/502 Data:0.001 | Batch:0.639 | Total:0:01:12 | ETA:0:04:14 | Loss:0.20554131799721503 | top1:91.

KeyboardInterrupt: 